In [2]:
import collections
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

In [3]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


#### what is the unit parameter for class LSTM?

In [23]:
# what is the unit parameter for class LSTM?
# see https://zhuanlan.zhihu.com/p/58854907
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
Tx = 30
n_x = 3
n_s = 64
X = Input(shape=(Tx, n_x))   
s, a, c = LSTM(n_s, return_sequences=True, return_state=True)(X)    
model_LSTM = Model(inputs=X, outputs=[s, a, c])
model_LSTM.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 30, 3)]           0         
_________________________________________________________________
lstm_16 (LSTM)               [(None, 30, 64), (None, 6 17408     
Total params: 17,408
Trainable params: 17,408
Non-trainable params: 0
_________________________________________________________________


In [15]:
model = tf.keras.Sequential()

Tx = 30
n_x = 3
n_s = 64

model.add(Input(shape=(Tx, n_x)))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128, return_sequences=True, return_state=False))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 30, 128)           67584     
_________________________________________________________________
dense_4 (Dense)              (None, 30, 10)            1290      
Total params: 68,874
Trainable params: 68,874
Non-trainable params: 0
_________________________________________________________________


In [32]:
Tx = 30
n_x = 3
n_s = 64*2
X = Input(shape=(Tx, n_x))   
s = LSTM(n_s, return_sequences=True, return_state=False)(X)    
d = Dense(10)(s)
model_LSTM = Model(inputs=X, outputs=[d])
model_LSTM.summary()


Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 30, 3)]           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 30, 128)           67584     
_________________________________________________________________
dense_8 (Dense)              (None, 30, 10)            1290      
Total params: 68,874
Trainable params: 68,874
Non-trainable params: 0
_________________________________________________________________


#### Topic: LSTM return_sequences

In [36]:
# Thema: return_sequences
# from https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

# 32 = batch_size
# 10 = 10 time steps
# 8  = 8 features
inputs = np.random.random([32, 10, 8]).astype(np.float32)
print(f"inputs: {inputs.shape}")
lstm = tf.keras.layers.LSTM(4)

output = lstm(inputs)  # The output has shape `[32, 4]`.
print(f"output: {output.shape}")

lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)

# whole_sequence_output has shape `[32, 10, 4]`.
# final_memory_state and final_carry_state both have shape `[32, 4]`.
whole_sequence_output, final_memory_state, final_carry_state = lstm(inputs)
print(f"whole_sequence_output: {whole_sequence_output.shape}, final_memory_state: {final_memory_state.shape}, final_carry_state: {final_carry_state.shape}")

inputs: (32, 10, 8)
output: (32, 4)
whole_sequence_output: (32, 10, 4), final_memory_state: (32, 4), final_carry_state: (32, 4)


#### Topic: LSTM return_sequences

In [51]:
# Thema: return_sequences
# https://www.dlology.com/blog/how-to-use-return_state-or-return_sequences-in-keras/
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
import keras

k_init = keras.initializers.Constant(value=0.1)
b_init = keras.initializers.Constant(value=0)
r_init = keras.initializers.Constant(value=0.1)
# LSTM units
units = 4

# define model
inputs1 = Input(shape=(3, 2))

lstm1 = LSTM(units, return_sequences=False, kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init)
print(f"lstm cell: {lstm1}")
lstm1 = lstm1(inputs1)
print(f"lstm1: {lstm1}")
model = Model(inputs=inputs1, outputs=lstm1)

# define input data
data = array([0.1, 0.2, 0.3, 0.1, 0.2, 0.3]).reshape((1,3,2))
# make and show prediction
output = model.predict(data)
print(f"output.shape: {output.shape}\noutput = {output}")

lstm cell: <keras.layers.recurrent.LSTM object at 0x000001F50BABA160>
lstm1: Tensor("lstm_14_1/strided_slice_18:0", shape=(None, 4), dtype=float32)
output.shape: (1, 4)
output = [[0.02765957 0.02765957 0.02765957 0.02765957]]


#### Time Series Windowing

see https://www.tensorflow.org/guide/data#time_series_windowing

In [53]:
range_ds = tf.data.Dataset.range(45)
batches = range_ds.batch(10, drop_remainder=True)

for batch in batches.take(5):
  print(batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]


In [58]:
def dense_1_step(batch, shift=1):
  # Shift features and labels one step relative to each other.
  return batch[:-shift], batch[shift:]

In [59]:
predict_dense_1_step = batches.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8]  =>  [1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18]  =>  [11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28]  =>  [21 22 23 24 25 26 27 28 29]


##### To predict a whole window instead of a fixed offset you can split the batches into two parts:

In [61]:
batches = range_ds.batch(15, drop_remainder=True)

def label_next_5_steps(batch):
  return (batch[:-5],   # Take all but the 5 last steps
          batch[-5:])   # take the remainder (5 steps)

predict_5_steps = batches.map(label_next_5_steps)

for features, label in predict_5_steps.take(3):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8 9]  =>  [10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]  =>  [25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]  =>  [40 41 42 43 44]


##### To allow some overlap between the features of one batch and the labels of another, use Dataset.zip:

In [62]:
feature_length = 10
label_length = 5

features = range_ds.batch(feature_length, drop_remainder=True)

labels = range_ds.batch(feature_length).skip(1).map(lambda labels: labels[:-5])

predict_5_steps = tf.data.Dataset.zip((features, labels))

for features, label in predict_5_steps.take(3):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8 9]  =>  [10 11 12 13 14]
[10 11 12 13 14 15 16 17 18 19]  =>  [20 21 22 23 24]
[20 21 22 23 24 25 26 27 28 29]  =>  [30 31 32 33 34]


In [63]:
window_size = 5

windows = range_ds.window(window_size, shift=1)
for sub_ds in windows.take(5):
  print(sub_ds)

<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>


In [66]:
 for x in windows.flat_map(lambda x: x).take(30):
   print(x.numpy(), end=' ')

Cause: could not parse the source code:

for x in windows.flat_map(lambda x: x).take(30):

This error may be avoided by creating the lambda in a standalone statement.

Cause: could not parse the source code:

for x in windows.flat_map(lambda x: x).take(30):

This error may be avoided by creating the lambda in a standalone statement.

0 1 2 3 4 1 2 3 4 5 2 3 4 5 6 3 4 5 6 7 4 5 6 7 8 5 6 7 8 9 

In [67]:
def sub_to_batch(sub):
  return sub.batch(window_size, drop_remainder=True)

for example in windows.flat_map(sub_to_batch).take(5):
  print(example.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]


In [69]:
def make_window_dataset(ds, window_size=5, shift=1, stride=1):
  windows = ds.window(window_size, shift=shift, stride=stride)

  def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

  windows = windows.flat_map(sub_to_batch)
  return windows


In [73]:
ds = make_window_dataset(range_ds, window_size=10, shift = 2, stride=1)

for example in ds.take(10):
  print(example.numpy())

[0 1 2 3 4 5 6 7 8 9]
[ 2  3  4  5  6  7  8  9 10 11]
[ 4  5  6  7  8  9 10 11 12 13]
[ 6  7  8  9 10 11 12 13 14 15]
[ 8  9 10 11 12 13 14 15 16 17]
[10 11 12 13 14 15 16 17 18 19]
[12 13 14 15 16 17 18 19 20 21]
[14 15 16 17 18 19 20 21 22 23]
[16 17 18 19 20 21 22 23 24 25]
[18 19 20 21 22 23 24 25 26 27]


In [77]:
# extract labels, as before:
dense_labels_ds = ds.map(lambda x: dense_1_step(x, shift=2))

for inputs,labels in dense_labels_ds.take(3):
  print(inputs.numpy(), "=>", labels.numpy())

[0 1 2 3 4 5 6 7] => [2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9] => [ 4  5  6  7  8  9 10 11]
[ 4  5  6  7  8  9 10 11] => [ 6  7  8  9 10 11 12 13]
